In [1]:
import os
import time
import random
# import numpy as np
import pandas as pd

import cv2 as cv
# import pygwalker as pyg

## 路径初始化

In [2]:
# dataset_folder = 'vr.net'
# projects_folders = [name for name in os.listdir(dataset_folder)]

project_folder = 'BeatSaber'

# 获取所有子文件夹的名称
sub_experiment_folders = [name for name in os.listdir(project_folder)]

print(sub_experiment_folders)

['P1 VRLOG-5041702']


In [3]:
# for sub in sub_experiment_folders:
#     data_path = f"{project_folder}/{sub}"

data_path = f"{project_folder}/{sub_experiment_folders[0]}"
print(data_path)

BeatSaber/P1 VRLOG-5041702


## 初始数据读取

In [4]:
def list_pic_files(path):
    '''
    读取指定目录下所有JPEG图片的名称，并返回它们的数量和名称。
    '''
    jpg_files = [f for f in os.listdir(path)]    # if f.endswith('.jpg')
    jpg_dict = {'File_Name': jpg_files}
    df = pd.DataFrame(jpg_dict)

    # 添加数据类型和文件类型两列
    df['Data_Type'] = df['File_Name'].apply(lambda x: x[0])
    df['File_Type'] = df['File_Name'].apply(lambda x: x[-3:])

    # 统计类型
    statistic_dic = {}
    for value in df['Data_Type'].unique():
        count = df[df['Data_Type'] == value].shape[0]
        statistic_dic[value] = [count]

    statistic_df = pd.DataFrame(statistic_dic)

    return df, statistic_df

In [11]:
pic_df, statistic_df = list_pic_files(data_path)
statistic_df.head()

,d,g,m,s,v
0,10346,1,10346,10346,1


## 不同类型数据分离

In [6]:
def extract_img_number(df: pd.DataFrame):
    df['File_Number'] = df.loc[:, ('File_Name')].apply(lambda x: int(x[1:x.find('.')]))
    df.sort_values(by='File_Number', inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

In [7]:
depth_df = df[df['Data_Type'] == 'd']
depth_df = extract_img_number(depth_df)
depth_df

C:\Users\joeyz\AppData\Local\Temp\ipykernel_31016\470591858.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['File_Number'] = df.loc[:, ('File_Name')].apply(lambda x: int(x[1:x.find('.')]))
C:\Users\joeyz\AppData\Local\Temp\ipykernel_31016\470591858.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values(by='File_Number', inplace=True)


,File_Name,Data_Type,File_Type,File_Number
0,d3.jpg,d,jpg,3
1,d6.jpg,d,jpg,6
2,d9.jpg,d,jpg,9
3,d12.jpg,d,jpg,12
4,d15.jpg,d,jpg,15
...,...,...,...,...
10341,d31026.jpg,d,jpg,31026
10342,d31029.jpg,d,jpg,31029
10343,d31032.jpg,d,jpg,31032
10344,d31035.jpg,d,jpg,31035


In [8]:
s_df = df[df['Data_Type'] == 's']
s_df = extract_img_number(s_df)
s_df

C:\Users\joeyz\AppData\Local\Temp\ipykernel_31016\470591858.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['File_Number'] = df.loc[:, ('File_Name')].apply(lambda x: int(x[1:x.find('.')]))
C:\Users\joeyz\AppData\Local\Temp\ipykernel_31016\470591858.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values(by='File_Number', inplace=True)


,File_Name,Data_Type,File_Type,File_Number
0,s3.jpg,s,jpg,3
1,s6.jpg,s,jpg,6
2,s9.jpg,s,jpg,9
3,s12.jpg,s,jpg,12
4,s15.jpg,s,jpg,15
...,...,...,...,...
10341,s31026.jpg,s,jpg,31026
10342,s31029.jpg,s,jpg,31029
10343,s31032.jpg,s,jpg,31032
10344,s31035.jpg,s,jpg,31035


## 视频写入

In [9]:
begin_img_name = s_df['File_Name'][0]
img = cv.imread(f"{data_path}/{begin_img_name}")
height, width, channels = img.shape
print(f"The First Image: {begin_img_name}, Shape: {img.shape}")

The First Image: s3.jpg, Shape: (1042, 2016, 3)


In [ ]:
def create_video_from_images(image_df, output_file, fps, width, height):
    """
    使用OpenCV按顺序从图片列表中合成视频。
    
    :param image_df: 包含图片文件名的DataFrame。
    :param output_file: 输出视频文件的路径。
    :param fps: 输出视频的帧率。
    :param width: 输出视频的宽度。
    :param height: 输出视频的高度。
    """
    # 定义视频编码器和创建VideoWriter对象
    fourcc = cv.VideoWriter_fourcc(*'mp4v')
    out = cv.VideoWriter(output_file, fourcc, fps, (width, height))
    
    for _, row in image_df.iterrows():
        img_path = row['File_Name']  # 假设DataFrame中的列名为'filename'
        img = cv.imread(f"{data_path}/{img_path}")
        if img is not None:
            # 调整图片大小以匹配输出视频的尺寸
            resized_img = cv.resize(img, (width, height))
            out.write(resized_img)
        else:
            print(f"Warning: Could not read image {img_path}")
    
    # 释放资源
    out.release()
    print(f"Video saved to {output_file}")

In [ ]:
# 对两大类图片数据分别生成视频
create_video_from_images(s_df, 's_output_video.mp4', fps=30, width=width, height=height)
create_video_from_images(depth_df, 'd_output_video.mp4', fps=30, width=width, height=height)